In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense
from keras.layers import Flatten, Convolution2D, MaxPooling2D, Conv2D
from keras.models import load_model
import os, re, glob
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, utils, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist
import shutil

In [ ]:
# 사진 모델 테스트
def Dataization(img_path):
    img = cv2.imread(img_path)
    return img / 256

src = []
name = []
test = []
image_dir = '/content/gdrive/MyDrive/인지론 프로젝트/dataset/test_data/'
for file in os.listdir(image_dir):
    if (file.find('.jpg') is not -1):
        src.append(image_dir + file)
        name.append(file)
        test.append(Dataization(image_dir + file))

In [ ]:
import matplotlib.image as img 
import matplotlib.pyplot as pp

test = np.array(test)
print(test.shape)
model = load_model('mask_hyun3.h5')
predict = model.predict(test)
print(predict.shape)
print("ImageName : , Predict : [mask, nomask]")
for i in range(len(test)):
    print(name[i] + " : , Predict : " + str(predict[i]))
    pp.imshow(test[i][:,:,::-1])
    pp.show()

    

In [ ]:
# 비디오 모델 테스트
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model

facenet = cv2.dnn.readNet('/content/gdrive/MyDrive/인지론 프로젝트/dataset/models/deploy.prototxt', '/content/gdrive/MyDrive/인지론 프로젝트/dataset/models/res10_300x300_ssd_iter_140000.caffemodel')
cap = cv2.VideoCapture('/content/gdrive/MyDrive/인지론 프로젝트/dataset/two_people.mov')
ret, img = cap.read()

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('/content/gdrive/MyDrive/인지론 프로젝트/dataset/two_people_detect(false22).mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (img.shape[1], img.shape[0]))

model = load_model('mask_hyun6.h5')

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    h, w = img.shape[:2]

    blob = cv2.dnn.blobFromImage(img, scalefactor=1., size=(300, 300), mean=(104., 177., 123.))
    facenet.setInput(blob)
    dets = facenet.forward()

    result_img = img.copy()

    for i in range(dets.shape[2]):
        confidence = dets[0, 0, i, 2]
        if confidence < 0.5:
            continue

        x1 = int(dets[0, 0, i, 3] * w)
        y1 = int(dets[0, 0, i, 4] * h)
        x2 = int(dets[0, 0, i, 5] * w)
        y2 = int(dets[0, 0, i, 6] * h)

        face = img[y1:y2, x1:x2]

        if (x2 >= w or y2 >= h):
            continue
        if (x1<=0 or y1<=0):
            continue

        #face_input = cv2.resize(face, dsize=(224, 224))
        face_input = cv2.resize(face,(200, 200))
        #face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
        #face_input = preprocess_input(face_input)
        face_input = np.expand_dims(face_input, axis=0)
        face_input = np.array(face_input)



        modelpredict = model.predict(face_input)
        mask=modelpredict[0][0]
        nomask=modelpredict[0][1]


        if mask > nomask:
            color = (0, 255, 0)
            label = 'Mask %d%%' % (mask * 100)
        else:
            color = (0, 0, 255)
            label = 'No Mask %d%%' % (nomask * 100)

        cv2.rectangle(result_img, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=color, lineType=cv2.LINE_AA)
        cv2.putText(result_img, text=label, org=(x1, y1 - 10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=color, thickness=2, lineType=cv2.LINE_AA)

    out.write(result_img)
    #cv2_imshow(result_img) # colab 환경에서 출력 때문에 계속 Busy 상태라 ignore
    if cv2.waitKey(1) == ord('q'):
        break

out.release()
cap.release()
